In [ ]:

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_classification
from itertools import product
from sklearn.model_selection import ParameterSampler
import random

In [2]:
import pandas as pd
train=pd.read_csv('../data/training_data.csv',low_memory=True, dtype=np.float32)
test=pd.read_csv('../data/testing_data.csv',low_memory=True, dtype=np.float32)



In [3]:
dep = ['Label']
X_train = train.loc[:,~train.columns.isin(dep)]
y_train = train.loc[:,train.columns.isin(dep)]

In [4]:
X_val = test.loc[:,~test.columns.isin(dep)]
y_val = test.loc[:,test.columns.isin(dep)]

In [5]:
# Apply MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_val)

In [6]:
# Define parameter grid
param_dist = {
    'C': np.logspace(-3, 2, 100),           # finer range for C
    'l1_ratio': np.linspace(0, 1, 100)      # finer range for l1_ratio
}

In [7]:
# Search for best parameters
best_auc = -1
best_params = {}
results = []
random.seed(42)
param_list = list(ParameterSampler(param_dist, n_iter=5, random_state=42))

In [ ]:
# Perform randomized search
for params in param_list:
    print(params)
    model = LogisticRegression(
        penalty='elasticnet',
        solver='saga',
        max_iter=1000,
        C=params['C'],
        l1_ratio=params['l1_ratio'],
        n_jobs=-1
    )
    model.fit(X_train_scaled, y_train)
    y_pred_prob = model.predict_proba(X_test_scaled)[:, 1]
    auc = roc_auc_score(y_val, y_pred_prob)
    print(auc)
    if auc > best_auc:
        print("Good")
        best_auc = auc
        best_params = params


c:\Users\ASUS\Desktop\Assignment - Sigmoid\model-dev\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5909249338377293
Good


c:\Users\ASUS\Desktop\Assignment - Sigmoid\model-dev\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5568347050816944


c:\Users\ASUS\Desktop\Assignment - Sigmoid\model-dev\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ASUS\Desktop\Assignment - Sigmoid\model-dev\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\ASUS\Desktop\Assignment - Sigmoid\model-dev\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5927469830342941
Good


c:\Users\ASUS\Desktop\Assignment - Sigmoid\model-dev\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\ASUS\Desktop\Assignment - Sigmoid\model-dev\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5930711241481941
Good


c:\Users\ASUS\Desktop\Assignment - Sigmoid\model-dev\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.5912775454871305


In [9]:
import joblib
best_model = LogisticRegression(
    penalty='elasticnet',
    solver='saga',
    max_iter=1000,
    C=best_params['C'],
    l1_ratio=best_params['l1_ratio'],
    n_jobs=-1
)
best_model.fit(X_train_scaled, y_train)

joblib.dump(best_model, '../model-objects/best_elasticnet_model.joblib')

c:\Users\ASUS\Desktop\Assignment - Sigmoid\model-dev\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ASUS\Desktop\Assignment - Sigmoid\model-dev\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


['../model-objects/best_elasticnet_model.joblib']

In [29]:
coefficients = best_model.coef_[0]
feature_names = [i for i in X_train.columns]

# Pair feature names with coefficients, filter out zero coefficients, and sort
sorted_nonzero_features = sorted(
    [(f, c) for f, c in zip(feature_names, coefficients) if c != 0],
    key=lambda x: abs(x[1]),
    reverse=False
)
sorted_nonzero_features

[('Var217_embed_10', np.float32(1.0589733e-08)),
 ('Var217_embed_11', np.float32(6.478925e-07)),
 ('Var217_embed_12', np.float32(1.3525032e-06)),
 ('Var217_embed_13', np.float32(1.6009573e-06)),
 ('Var217_embed_14', np.float32(0.00013143923)),
 ('Var197_embed_32', np.float32(-0.0009926959)),
 ('Var217_embed_15', np.float32(0.0011832234)),
 ('Var197_embed_31', np.float32(-0.0012995516)),
 ('Var197_embed_30', np.float32(-0.0014324508)),
 ('Var217_embed_16', np.float32(0.0017367547)),
 ('Var197_embed_29', np.float32(-0.0020858592)),
 ('Var197_embed_28', np.float32(-0.0025874046)),
 ('Var217_embed_17', np.float32(0.0026669558)),
 ('Var197_embed_27', np.float32(-0.0038885057)),
 ('Var217_embed_18', np.float32(0.005564601)),
 ('Var197_embed_26', np.float32(-0.005822561)),
 ('Var197_embed_25', np.float32(-0.00649416)),
 ('Var217_embed_19', np.float32(0.007772958)),
 ('Var217_embed_20', np.float32(0.009381813)),
 ('Var197_embed_24', np.float32(-0.010209796)),
 ('Var217_embed_21', np.float32(0.

In [30]:
def ks(data=None,target=None, prob=None):
    data['target0'] = 1 - data[target]
    data['bucket'] = pd.qcut(data[prob], 10)
    grouped = data.groupby('bucket', as_index = False)
    kstable = pd.DataFrame()
    kstable['min_prob'] = grouped.min()[prob]
    kstable['max_prob'] = grouped.max()[prob]
    kstable['events']   = grouped.sum()[target]
    kstable['nonevents'] = grouped.sum()['target0']
    kstable = kstable.sort_values(by="min_prob", ascending=False).reset_index(drop = True)
    kstable['event_rate'] = (kstable.events / data[target].sum()).apply('{0:.2%}'.format)
    kstable['nonevent_rate'] = (kstable.nonevents / data['target0'].sum()).apply('{0:.2%}'.format)
    kstable['cum_eventrate']=(kstable.events / data[target].sum()).cumsum()
    kstable['cum_noneventrate']=(kstable.nonevents / data['target0'].sum()).cumsum()
    kstable['KS'] = np.round(kstable['cum_eventrate']-kstable['cum_noneventrate'], 3) * 100

    #Formating
    kstable['cum_eventrate']= kstable['cum_eventrate'].apply('{0:.2%}'.format)
    kstable['cum_noneventrate']= kstable['cum_noneventrate'].apply('{0:.2%}'.format)
    kstable.index = range(1,11)
    kstable.index.rename('Decile', inplace=True)
    pd.set_option('display.max_columns', 9)
    print(kstable)
    
    #Display KS
    from colorama import Fore
    print(Fore.RED + "KS is " + str(max(kstable['KS']))+"%"+ " at decile " + str((kstable.index[kstable['KS']==max(kstable['KS'])][0])))
    return(kstable)

In [31]:
y_train_prob = best_model.predict_proba(X_train_scaled)[:, 1]
y_val_prob = best_model.predict_proba(X_test_scaled)[:, 1]
data_ap_train = pd.DataFrame({'Actual':y_train.values.ravel(),'Predicted':y_train_prob})
data_ap_val = pd.DataFrame({'Actual':y_val.values.ravel(),'Predicted':y_val_prob})

In [32]:
ks_train=ks(data=data_ap_train,target='Actual',prob='Predicted')


        min_prob  max_prob  events  nonevents event_rate nonevent_rate  \
Decile                                                                   
1       0.702723  0.993325   222.0     3278.0      8.65%        10.11%   
2       0.551030  0.702672   228.0     3272.0      8.89%        10.09%   
3       0.442659  0.551002   233.0     3267.0      9.08%        10.07%   
4       0.356152  0.442658   259.0     3241.0     10.09%         9.99%   
5       0.282739  0.356123   241.0     3259.0      9.39%        10.05%   
6       0.215690  0.282734   289.0     3211.0     11.26%         9.90%   
7       0.158718  0.215688   264.0     3236.0     10.29%         9.98%   
8       0.108263  0.158713   273.0     3227.0     10.64%         9.95%   
9       0.060005  0.108263   292.0     3208.0     11.38%         9.89%   
10      0.001301  0.059987   265.0     3235.0     10.33%         9.97%   

       cum_eventrate cum_noneventrate   KS  
Decile                                      
1              8.65% 

C:\Users\ASUS\AppData\Local\Temp\ipykernel_14904\572846000.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = data.groupby('bucket', as_index = False)


In [33]:
ks_val=ks(data=data_ap_val,target='Actual',prob='Predicted')

        min_prob  max_prob  events  nonevents event_rate nonevent_rate  \
Decile                                                                   
1       0.710201  0.987850    97.0     1403.0      8.77%        10.10%   
2       0.557202  0.710177   101.0     1399.0      9.13%        10.07%   
3       0.452328  0.557145   100.0     1400.0      9.04%        10.08%   
4       0.364357  0.452256   101.0     1399.0      9.13%        10.07%   
5       0.288103  0.364201   114.0     1386.0     10.31%         9.98%   
6       0.220214  0.288065   128.0     1372.0     11.57%         9.87%   
7       0.161310  0.220176   118.0     1382.0     10.67%         9.95%   
8       0.108730  0.161305   139.0     1361.0     12.57%         9.80%   
9       0.060943  0.108708   108.0     1392.0      9.76%        10.02%   
10      0.001522  0.060916   100.0     1400.0      9.04%        10.08%   

       cum_eventrate cum_noneventrate   KS  
Decile                                      
1              8.77% 

C:\Users\ASUS\AppData\Local\Temp\ipykernel_14904\572846000.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = data.groupby('bucket', as_index = False)
